In [2]:
import time
import numpy as np
import pandas as pd

In [3]:
df_train = pd.read_csv('research_train_set.csv')
num_dates = df_train["date_id"].nunique()  # this number should be 401
num_seconds_in_bucket = df_train["seconds_in_bucket"].nunique()  # this number should be 55
col2index_map = {key: value for (key, value) in zip(df_train.columns, range(len(df_train.columns)))}

In [4]:
def factor(current_data: np.ndarray, hist_list: list) -> np.ndarray:
    '''
    This will be the main function to design your factors for the competition. Please
    define only one factor here each time. We provide you with:

    current_data: The numpy array that contains the data up to the current date_id and
    seconds_in_bucket in the loop

    hist_list: A list for you to save the previous factor values (optional). For instance,
    if you are calculating a 100-day Moving Averge (MA), then you can save the first calculated
    MA in hist_list, and then for the next MA calculation, you can use the saved ones.
    '''
    ###################### ADD YOUR CODE HERE FOR FACTORS DESIGN ######################
    res = current_data[:, [col2index_map['stock_id'], col2index_map['date_id'], col2index_map['seconds_in_bucket']]]
    res = current_data[:, col2index_map['ask_price']] - current_data[:, col2index_map['bid_price']]
    return res  # The return value MUSE BE a numpy array

# Original version

In [5]:
hist_list = []
# tongfei's version
all_test = np.empty((200*num_seconds_in_bucket*2000,16))
final_factor_value = np.empty((200*num_seconds_in_bucket*2000,))
current_row = 0
for date_id in range(num_dates):
    query_time = 0.
    paste_time = 0.
    factor_time = 0.
    for seconds_in_bucket in range(num_seconds_in_bucket):
        seconds_in_bucket *= 10
        # part 1: query data
        time_start_query = time.time()
        new_test_data = np.array(df_train
                                .query(f'date_id == {date_id} & seconds_in_bucket == {seconds_in_bucket}')
                                .drop(columns = ["target"])
                                .reset_index(drop=True))
        time_end_query = time.time()
        # part 2: paste data to all_test
        time_paste_data_start = time.time()
        all_test[current_row:current_row+len(new_test_data), :] = new_test_data
        time_paste_data_end = time.time()
        # part 3: calculate factor
        
        time_factor_calc_start = time.time()
        final_factor_value[current_row:current_row+len(new_test_data),] = factor(current_data=all_test[current_row:current_row+len(new_test_data),:], 
                                                                                 hist_list=hist_list)
        time_factor_calc_end = time.time()
        
        query_time += time_end_query - time_start_query
        paste_time += time_paste_data_end - time_paste_data_start
        factor_time += time_factor_calc_end - time_factor_calc_start
    break

In [6]:
# time expense for 1 day
print(f'query time: {query_time}, {query_time/(query_time+paste_time+factor_time)*100}%')
print(f'paste time: {paste_time}, {paste_time/(query_time+paste_time+factor_time)*100}%')
print(f'factor time: {factor_time}, {factor_time/(query_time+paste_time+factor_time)*100}%')
print(f'predicted_process_time: {(query_time+paste_time+factor_time)*400/60} minutes')

query time: 1.8800549507141113, 99.47396452376381%
paste time: 0.008942604064941406, 0.4731544039008873%
factor time: 0.00099945068359375, 0.05288107233530233%
predicted_process_time: 12.599980036417643 minutes


# It seems like the query is the problem. query tool is sloooow. using native pandas slicer

In [7]:
hist_list = []
# kevin's readability version
all_test = np.empty((200*num_seconds_in_bucket*2000,16))
final_factor_value = np.empty((200*num_seconds_in_bucket*2000,))
current_row = 0
for date_id in range(num_dates):
    query_time = 0.
    paste_time = 0.
    factor_time = 0.
    for seconds_in_bucket in range(num_seconds_in_bucket):
        seconds_in_bucket *= 10
        # part 1: query data
        time_start_query = time.time()
        new_test_data = df_train[(df_train['date_id'] == date_id) & (df_train['seconds_in_bucket'] == seconds_in_bucket)].drop(columns = ["target"]).reset_index(drop=True)
        time_end_query = time.time()
        # part 2: paste data to all_test
        time_paste_data_start = time.time()
        all_test[current_row:current_row+len(new_test_data), :] = new_test_data
        time_paste_data_end = time.time()
        # part 3: calculate factor
        
        time_factor_calc_start = time.time()
        final_factor_value[current_row:current_row+len(new_test_data),] = factor(current_data=all_test[current_row:current_row+len(new_test_data),:], 
                                                                                 hist_list=hist_list)
        time_factor_calc_end = time.time()
        
        query_time += time_end_query - time_start_query
        paste_time += time_paste_data_end - time_paste_data_start
        factor_time += time_factor_calc_end - time_factor_calc_start
    print(new_test_data)
    break

     stock_id  date_id  seconds_in_bucket  imbalance_size  \
0           0        0                540       906957.07   
1           1        0                540        92236.28   
2           2        0                540            0.00   
3           3        0                540      2446727.30   
4           4        0                540            0.00   
..        ...      ...                ...             ...   
186       194        0                540       583278.56   
187       195        0                540       304130.06   
188       196        0                540      1595817.99   
189       197        0                540       354616.47   
190       198        0                540      4357623.66   

     imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                          1         0.999383   27183793.68   1.000241   
1                         -1         1.000425    3439290.16   0.999955   
2                          0         0.999877

In [8]:
# time expense for 1 day
print(f'query time: {query_time}, {query_time/(query_time+paste_time+factor_time)*100}%')
print(f'paste time: {paste_time}, {paste_time/(query_time+paste_time+factor_time)*100}%')
print(f'factor time: {factor_time}, {factor_time/(query_time+paste_time+factor_time)*100}%')
print(f'predicted_process_time: {(query_time+paste_time+factor_time)*400/60} minutes')

query time: 0.8770031929016113, 98.42784865133315%
paste time: 0.011001348495483398, 1.234703674329145%
factor time: 0.0030066967010498047, 0.3374476743377078%
predicted_process_time: 5.940074920654297 minutes


# We can also be one step more efficient by removing the overhead altogether
idea from https://stackoverflow.com/questions/57208997/looking-for-the-fastest-way-to-slice-a-row-in-a-huge-pandas-dataframe 
and 
https://stackoverflow.com/questions/49222788/converting-pandas-dataframe-to-dictionary-using-index-option-with-a-non-unique

In [9]:
overhead_start = time.time()
df_train['slice_index'] = df_train['date_id'].astype(str) + '_' + df_train['seconds_in_bucket'].astype(str)  # 3 seconds
dic_sorted = df_train.drop(columns=['target']).groupby('slice_index').agg(lambda x: x.tolist()).to_dict('index')  # 1min
overhead_end = time.time()
overhead_time = overhead_end - overhead_start

In [10]:
hist_list = []
# kevin's readability version
all_test = np.empty((200*num_seconds_in_bucket*2000,16))
final_factor_value = np.empty((200*num_seconds_in_bucket*2000,))
current_row = 0
for date_id in range(num_dates):
    query_time = 0.
    paste_time = 0.
    factor_time = 0.
    for seconds_in_bucket in range(num_seconds_in_bucket):
        seconds_in_bucket *= 10
        # part 1: query data
        time_start_query = time.time()
        new_test_data = pd.DataFrame(dic_sorted[f'{date_id}_{seconds_in_bucket}'])
        time_end_query = time.time()
        # part 2: paste data to all_test
        time_paste_data_start = time.time()
        all_test[current_row:current_row+len(new_test_data), :] = new_test_data
        time_paste_data_end = time.time()
        # part 3: calculate factor
        
        time_factor_calc_start = time.time()
        final_factor_value[current_row:current_row+len(new_test_data),] = factor(current_data=all_test[current_row:current_row+len(new_test_data),:], 
                                                                                 hist_list=hist_list)
        time_factor_calc_end = time.time()
        
        query_time += time_end_query - time_start_query
        paste_time += time_paste_data_end - time_paste_data_start
        factor_time += time_factor_calc_end - time_factor_calc_start
    print(new_test_data)
    break

     stock_id  date_id  seconds_in_bucket  imbalance_size  \
0           0        0                540       906957.07   
1           1        0                540        92236.28   
2           2        0                540            0.00   
3           3        0                540      2446727.30   
4           4        0                540            0.00   
..        ...      ...                ...             ...   
186       194        0                540       583278.56   
187       195        0                540       304130.06   
188       196        0                540      1595817.99   
189       197        0                540       354616.47   
190       198        0                540      4357623.66   

     imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                          1         0.999383   27183793.68   1.000241   
1                         -1         1.000425    3439290.16   0.999955   
2                          0         0.999877

In [13]:
# time expense for 1 day
print(f'overhead time: {overhead_time} seconds')
print(f'query time: {query_time}, {query_time/(query_time+paste_time+factor_time)*100}%')
print(f'paste time: {paste_time}, {paste_time/(query_time+paste_time+factor_time)*100}%')
print(f'factor time: {factor_time}, {factor_time/(query_time+paste_time+factor_time)*100}%')
print(f'predicted_process_time: {(query_time+paste_time+factor_time)*400/60} minutes')

overhead time: 22.133418083190918 seconds
query time: 0.05000185966491699, 81.9717175823145%
paste time: 0.008998394012451172, 14.751727588255525%
factor time: 0.0019986629486083984, 3.2765548294299744%
predicted_process_time: 0.40665944417317706 minutes
